## Packages

In [8]:
install.packages("car")
install.packages("dplyr")
install.packages("haven")
install.packages("lfe")
install.packages("sandwich")
install.packages("plm")
install.packages("stargazer")

Installing package into ‘/home/preston/R/x86_64-pc-linux-gnu-library/4.4’
(as ‘lib’ is unspecified)



Installing package into ‘/home/preston/R/x86_64-pc-linux-gnu-library/4.4’
(as ‘lib’ is unspecified)

Installing package into ‘/home/preston/R/x86_64-pc-linux-gnu-library/4.4’
(as ‘lib’ is unspecified)

Installing package into ‘/home/preston/R/x86_64-pc-linux-gnu-library/4.4’
(as ‘lib’ is unspecified)

also installing the dependencies ‘Rcpp’, ‘miscTools’, ‘generics’, ‘rbibutils’, ‘bdsmatrix’, ‘collapse’, ‘lmtest’, ‘maxLik’, ‘Rdpack’




In [9]:
library(haven)
library(lfe)
library(lmtest)
library(dplyr)
library(stargazer)


Please cite as: 


 Hlavac, Marek (2022). stargazer: Well-Formatted Regression and Summary Statistics Tables.

 R package version 5.2.3. https://CRAN.R-project.org/package=stargazer 




## Table 2

Column 1

In [48]:
performance_during_exper <- haven::read_dta("data/performance_during_exper.dta")

# Create a categorical variable for year_week
performance_during_exper$year_week <- as.factor(performance_during_exper$year_week)

# Fit the regression model
model <- felm(formula = perform1 ~ experiment_treatment + year_week |
                personid | 0 | (1 | personid),
              data = performance_num_or_log,
              subset = (year_week != 201049 & (expgroup == 1 | expgroup == 0)),
              weights = NULL)

# Print the summary of the model
summary(model)

# Check for heteroskedasticity
coeftest(model, vcov = vcovHC, type = "HC1")

Warning message in pt(abs(z$STATS[[lhs]]$tval), z$df, lower.tail = FALSE):
“NaNs produced”
Warning message in pt(abs(z$STATS[[lhs]]$rtval), z$df, lower.tail = FALSE):
“NaNs produced”
Warning message in chol.default(mat, pivot = TRUE, tol = tol):
“the matrix is either rank-deficient or not positive definite”


Error in chol.default(mat[ok, ok]) : 'a' must have dims > 0


Warning message in summary.felm(model):
“can't compute cluster F-test”



Call:
   felm(formula = perform1 ~ experiment_treatment + year_week |      personid | 0 | (1 | personid), data = performance_num_or_log,      subset = (year_week != 201049 & (expgroup == 1 | expgroup ==          0)), weights = NULL) 

Residuals:
    Min      1Q  Median      3Q     Max 
-4.2689 -0.4187  0.0424  0.4687  3.7219 

Coefficients:
                     Estimate Cluster s.e. t value Pr(>|t|)
experiment_treatment   0.2324          NaN     NaN      NaN
year_week201002        1.6294          NaN     NaN      NaN
year_week201003        1.7695          NaN     NaN      NaN
year_week201004        1.8743          NaN     NaN      NaN
year_week201005        1.7777          NaN     NaN      NaN
year_week201006        1.7660          NaN     NaN      NaN
year_week201007        1.5339          NaN     NaN      NaN
year_week201008        1.6779          NaN     NaN      NaN
year_week201009        1.6264          NaN     NaN      NaN
year_week201010        1.7321          NaN     NaN      

Warning message in chol.default(mat, pivot = TRUE, tol = tol):
“the matrix is either rank-deficient or not positive definite”


Error in chol.default(mat[ok, ok]) : 'a' must have dims > 0


Warning message in summary.felm(x):
“can't compute cluster F-test”



z test of coefficients:

                     Estimate Std. Error z value Pr(>|z|)
experiment_treatment  0.23236        NaN     NaN      NaN
year_week201002       1.62942        NaN     NaN      NaN
year_week201003       1.76952        NaN     NaN      NaN
year_week201004       1.87433        NaN     NaN      NaN
year_week201005       1.77772        NaN     NaN      NaN
year_week201006       1.76600        NaN     NaN      NaN
year_week201007       1.53386        NaN     NaN      NaN
year_week201008       1.67786        NaN     NaN      NaN
year_week201009       1.62635        NaN     NaN      NaN
year_week201010       1.73211        NaN     NaN      NaN
year_week201011       1.73962        NaN     NaN      NaN
year_week201012       1.78847        NaN     NaN      NaN
year_week201013       1.86223        NaN     NaN      NaN
year_week201014       1.80623        NaN     NaN      NaN
year_week201015       1.63815        NaN     NaN      NaN
year_week201016       1.96137        NaN     N

In [49]:
# num_or_log_cols <- names(performance_during_exper)[sapply(performance_during_exper, function(x) is.numeric(x) || is.logical(x))]
performance_num_or_log <- performance_during_exper[, c("personid", "expgroup", "perform1", "treatment", "experiment_treatment", "experiment_control", "experiment_nantong", "homethatweek", "experiment_home", "wage_month")]
cov(performance_num_or_log)

,personid,expgroup,perform1,treatment,experiment_treatment,experiment_control,experiment_nantong,homethatweek,experiment_home,wage_month
personid,1.624687e+09,3.245444e+04,NA,-5.767060e+03,-2.397190e+03,-1.917044e+03,6197.55870675,1.446933e+04,4.065864e+03,-2.883755e+04
expgroup,3.245444e+04,8.900378e-01,NA,-1.356709e-01,-5.631925e-02,-7.283009e-02,0.14751445,3.204519e-01,9.784447e-02,1.665750e-01
perform1,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
treatment,-5.767060e+03,-1.356709e-01,NA,8.230866e-02,3.416769e-02,-2.637282e-03,-0.02665412,-3.651839e-02,2.937024e-03,1.855303e-01
experiment_treatment,-2.397190e+03,-5.631925e-02,NA,3.416769e-02,3.615616e-02,-1.094780e-03,-0.01106457,3.393198e-03,2.027462e-02,1.991644e+00
experiment_control,-1.917044e+03,-7.283009e-02,NA,-2.637282e-03,-1.094780e-03,2.829271e-02,-0.00858304,-2.233794e-02,-9.242630e-03,1.503873e+00
experiment_nantong,6.197559e+03,1.475144e-01,NA,-2.665412e-02,-1.106457e-02,-8.583040e-03,0.20778198,6.570538e-02,1.980552e-01,1.530171e+01
homethatweek,1.446933e+04,3.204519e-01,NA,-3.651839e-02,3.393198e-03,-2.233794e-02,0.06570538,1.733208e-01,7.307287e-02,1.838632e+00
experiment_home,4.065864e+03,9.784447e-02,NA,2.937024e-03,2.027462e-02,-9.242630e-03,0.19805523,7.307287e-02,2.202630e-01,1.704509e+01
wage_month,-2.883755e+04,1.665750e-01,NA,1.855303e-01,1.991644e+00,1.503873e+00,15.30170892,1.838632e+00,1.704509e+01,2.144006e+03


In [18]:
names(performance_during_exper)

[1] "personid"                        "year_week"                      
 [3] "expgroup"                        "perform1"                       
 [5] "phonecall"                       "phonecallraw"                   
 [7] "treatment"                       "experiment_treatment"           
 [9] "experiment_control"              "experiment_nantong"             
[11] "homethatweek"                    "experiment_home"                
[13] "logphonecall"                    "logcallpersec"                  
[15] "logcalllength"                   "logcall_dayworked"              
[17] "logdaysworked"                   "wage_month"                     
[19] "experiment_treatment_commute120" "date"

In [20]:
length(unique(performance_during_exper$expgroup))

[1] 4

Column 2

## Table 4

In [17]:
performance_during_exper <- haven::read_dta("data/performance_during_exper.dta")

performance_during_exper_excluding_2 <- performance_during_exper %>%
  filter(year_week != 201049, expgroup != 2)

performance_during_exper_excluding_3 <- performance_during_exper %>%
  filter(year_week != 201049, expgroup != 3)

model_perform1_excluding_2 <- felm(
  formula = perform1 ~ experiment_treatment + experiment_control + factor(year_week) |
  personid | 0 | 0 | (1 | personid),
  data = performance_during_exper_excluding_2
)

model_phonecall_excluding_2 <- felm(
  formula = phonecall ~ experiment_treatment + experiment_control + factor(year_week) |
  personid | 0 | 0 | (1 | personid),
  data = performance_during_exper_excluding_2
)

model_perform1_excluding_3 <- felm(
  formula = perform1 ~ experiment_treatment + experiment_control + factor(year_week) |
  personid | 0 | 0 | (1 | personid),
  data = performance_during_exper_excluding_3
)

model_phonecall_excluding_3 <- felm(
  formula = phonecall ~ experiment_treatment + experiment_control + factor(year_week) |
  personid | 0 | 0 | (1 | personid),
  data = performance_during_exper_excluding_3
)

cat("perform1 excluding group 2")
summary(model_perform1_excluding_2)

cat("phonecall excluding group 2")
summary(model_phonecall_excluding_2)

cat("perform1 excluding group 3")
summary(model_perform1_excluding_3)

cat("phonecall excluding group 3")
summary(model_phonecall_excluding_3)

perform1 excluding group 2


Call:
   felm(formula = perform1 ~ experiment_treatment + experiment_control +      factor(year_week) | personid | 0 | 0 | (1 | personid), data = performance_during_exper_excluding_2) 

Residuals:
    Min      1Q  Median      3Q     Max 
-4.8705 -0.4650  0.0404  0.5166  3.9519 

Coefficients:
                        Estimate Std. Error t value Pr(>|t|)    
experiment_treatment     0.19396    0.01796  10.797   <2e-16 ***
experiment_control      -0.03467    0.01975  -1.755   0.0792 .  
factor(year_week)201002  1.80792    0.04601  39.296   <2e-16 ***
factor(year_week)201003  2.06448    0.04599  44.892   <2e-16 ***
factor(year_week)201004  2.15205    0.04602  46.767   <2e-16 ***
factor(year_week)201005  2.21217    0.04603  48.059   <2e-16 ***
factor(year_week)201006  2.11384    0.04594  46.012   <2e-16 ***
factor(year_week)201007  1.38370    0.04617  29.968   <2e-16 ***
factor(year_week)201008  1.99012    0.04620  43.078   <2e-16 ***
factor(year_week)201009  1.66492    0.04576  36.383   <

phonecall excluding group 2


Call:
   felm(formula = phonecall ~ experiment_treatment + experiment_control +      factor(year_week) | personid | 0 | 0 | (1 | personid), data = performance_during_exper_excluding_2) 

Residuals:
    Min      1Q  Median      3Q     Max 
-4.8708 -0.4535  0.0516  0.5169  4.4633 

Coefficients:
                        Estimate Std. Error t value Pr(>|t|)    
experiment_treatment     0.28073    0.02371  11.840   <2e-16 ***
experiment_control      -0.01091    0.02672  -0.408    0.683    
factor(year_week)201002  1.95374    0.05116  38.190   <2e-16 ***
factor(year_week)201003  2.21211    0.05114  43.254   <2e-16 ***
factor(year_week)201004  2.31134    0.05119  45.156   <2e-16 ***
factor(year_week)201005  2.36769    0.05120  46.247   <2e-16 ***
factor(year_week)201006  2.27925    0.05107  44.630   <2e-16 ***
factor(year_week)201007  1.55329    0.05136  30.242   <2e-16 ***
factor(year_week)201008  2.32007    0.05134  45.192   <2e-16 ***
factor(year_week)201009  1.81709    0.05085  35.733   

perform1 excluding group 3


Call:
   felm(formula = perform1 ~ experiment_treatment + experiment_control +      factor(year_week) | personid | 0 | 0 | (1 | personid), data = performance_during_exper_excluding_3) 

Residuals:
    Min      1Q  Median      3Q     Max 
-4.3001 -0.4253  0.0384  0.4730  3.7797 

Coefficients:
                        Estimate Std. Error t value Pr(>|t|)    
experiment_treatment     0.30153    0.02360  12.775  < 2e-16 ***
experiment_control       0.06634    0.02487   2.668  0.00764 ** 
factor(year_week)201002  1.68219    0.07495  22.443  < 2e-16 ***
factor(year_week)201003  1.79349    0.07519  23.853  < 2e-16 ***
factor(year_week)201004  1.88997    0.07490  25.232  < 2e-16 ***
factor(year_week)201005  1.83249    0.07486  24.478  < 2e-16 ***
factor(year_week)201006  1.83239    0.07475  24.514  < 2e-16 ***
factor(year_week)201007  1.50369    0.07464  20.145  < 2e-16 ***
factor(year_week)201008  1.74151    0.07495  23.235  < 2e-16 ***
factor(year_week)201009  1.64357    0.07476  21.986  < 

phonecall excluding group 3


Call:
   felm(formula = phonecall ~ experiment_treatment + experiment_control +      factor(year_week) | personid | 0 | 0 | (1 | personid), data = performance_during_exper_excluding_3) 

Residuals:
    Min      1Q  Median      3Q     Max 
-4.2909 -0.3690  0.0782  0.4828  4.6398 

Coefficients:
                        Estimate Std. Error t value Pr(>|t|)    
experiment_treatment     0.31206    0.03262   9.567   <2e-16 ***
experiment_control       0.01919    0.03477   0.552    0.581    
factor(year_week)201002  2.05485    0.10021  20.505   <2e-16 ***
factor(year_week)201003  2.13822    0.10049  21.277   <2e-16 ***
factor(year_week)201004  2.33876    0.10002  23.382   <2e-16 ***
factor(year_week)201005  2.19691    0.09976  22.022   <2e-16 ***
factor(year_week)201006  2.25662    0.09967  22.640   <2e-16 ***
factor(year_week)201007  1.77515    0.09977  17.793   <2e-16 ***
factor(year_week)201008  2.25094    0.09984  22.546   <2e-16 ***
factor(year_week)201009  2.00701    0.09959  20.153   